In [1]:
import pandas as pd
import plotly 
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

In [2]:
#register here to get api_key https://plot.ly/settings/api#/
plotly.tools.set_credentials_file(username='myvarinfo', api_key='Og4r4YyrYkP6HJ74k53o')

In [21]:
df = pd.read_csv("https://ftp.ncbi.nih.gov/snp/temp/Demo_Data/ClinVarSNP_Demo3.csv")

In [24]:
df.head()

,CHR,POS,SNP_ID,MAF_1000G,MAF_ExAC,MAF_TOPMED,MAF_ESP,GENE,GENE_GROUP
0,1,1014358,672601312.0,NaN,0.000008,NaN,NaN,ISG15,GTR
1,1,1014358,672601312.0,NaN,0.000008,NaN,NaN,ISG15,CGD
2,1,1014358,672601312.0,NaN,0.000008,NaN,NaN,ISG15,CLINVAR
3,1,1022224,756623659.0,NaN,0.000008,NaN,NaN,AGRN,GTR
4,1,1022224,756623659.0,NaN,0.000008,NaN,NaN,AGRN,CGD


In [27]:
print(df.GENE_GROUP.unique())

['GTR' 'CGD' 'CLINVAR' 'INVITAE' 'COSMIC' 'PMID:26627737' 'ACMG']


In [42]:
acmg_var =  df.loc[(df['GENE_GROUP'] == 'ACMG') & (df['MAF_TOPMED'] > 0) ]

In [46]:
print ("row_count = ", acmg_var.shape[0])
acmg_var.head()

row_count =  376


,CHR,POS,SNP_ID,MAF_1000G,MAF_ExAC,MAF_TOPMED,MAF_ESP,GENE,GENE_GROUP
352,1,17022647,74315368.0,NaN,0.000025,0.000024,NaN,SDHB,ACMG
373,1,17024014,397516836.0,NaN,NaN,0.000008,NaN,SDHB,ACMG
378,1,17027801,33927012.0,0.009385,0.012545,0.007900,0.009534,SDHB,ACMG
399,1,17044823,772551056.0,NaN,0.000008,0.000024,NaN,SDHB,ACMG
431,1,17053946,587782703.0,NaN,NaN,0.000008,NaN,SDHB,ACMG


In [47]:
acmg_var.groupby(['CHR', 'GENE']).size()

CHR  GENE  
1    LMNA       7
     MUTYH     31
     RYR2       1
     SDHB       6
     SDHC       2
     TNNT2      5
10   RET        8
11   KCNQ1     18
     MEN1       1
     MYBPC3    26
     SDHD       3
12   MYL2       2
     PKP2      13
13   ATP7B     19
     BRCA2     29
     RB1        2
14   MYH7      17
16   MYH11      1
     TSC2       2
17   BRCA1     25
     TP53      12
18   DSC2       2
     DSG2       3
     SMAD4      1
19   LDLR       9
     RYR1      22
     TNNI3      5
2    APOB       4
     COL3A1     1
     MSH2       2
     MSH6       8
3    MLH1       2
     MYL3       2
     SCN5A     36
     TGFBR2     2
     TMEM43     1
     VHL        3
5    APC        4
6    DSP        4
7    KCNH2      8
     PMS2      19
X    GLA        4
     OTC        4
dtype: int64

In [33]:
trace = go.Scatter(
    x = acmg_var['MAF_1000G'],
    y = acmg_var['MAF_TOPMED'],
    mode = 'markers'
)
data = [trace]

In [52]:
layout = dict(title = 'MAF 1000G vs TOPMED',
              autosize=False,
              width=600,
              height=600,
              xaxis = dict(title= '1000G', range=[0, 0.01]),
              yaxis = dict(title= 'TOPMED', range=[0, 0.01]),
             )

In [53]:
fig = dict(data=data, layout=layout)
# Scatter of 1000G vs TOPMED MAF
py.iplot(fig, filename='MAF_1000G_TOPMED')